# MAST30034_Applied Data Science_Project1

## Import Libraries

In [32]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from geodatasets import get_path
import pandas as pd
import geopandas as gpd
from pyspark.sql.functions import date_format, hour, dayofweek
import seaborn as sns
from scipy.stats import chi2_contingency
from pyspark.sql.functions import count as spark_count
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [33]:
# Create a spark session
spark = (
SparkSession.builder.appName("ADS project 1")
.config("spark.sql.repl.eagerEval.enabled", True)
.config("spark.driver.memory","6G")
.config("spark.executor.memory","6G")
.config("spark.sql.parquet.cacheMetadata", "true")
.getOrCreate()
)

## Read the data

In [34]:
taxi_data_9 = spark.read.parquet('../data/after_EDA_data_Engineering/taxi-9').toPandas()
citybike_data_9 = spark.read.parquet('../data/after_EDA_data_Engineering/citybike-9').toPandas()
taxi_data_2 = spark.read.parquet('../data/after_EDA_data_Engineering/taxi-2').toPandas()
citybike_data_2 = spark.read.parquet('../data/after_EDA_data_Engineering/citybike-2').toPandas()

In [35]:
taxi_data_9.head()

,hour,temp,humidity,preciptype,windgust,visibility,solarenergy,icon,day_of_week,taxi_count
0,5,12.1,69.17,not rain,19.5,9.9,0,partly-cloudy-night,Sun,545
1,23,19.3,72.82,not rain,4.7,9.9,0,cloudy,Fri,5553
2,15,25,50.22,not rain,16.1,9.9,2.7,clear-day,Tue,6321
3,18,26.1,66.61,not rain,10.3,9.9,1.5,clear-day,Sat,6098
4,0,21.7,89.94,not rain,6.9,9.8,0,clear-night,Mon,2329


In [36]:
taxi_data_2.head()

,hour,temp,humidity,preciptype,windgust,visibility,solarenergy,icon,day_of_week,taxi_count
0,11,10.6,58.15,not rain,18.3,9.9,1.9,clear-day,Fri,5523
1,6,-1.7,67.89,not rain,18.3,9.9,0,clear-night,Wed,1586
2,12,12.8,49.81,not rain,45,9.9,1.9,clear-day,Sun,5764
3,11,9.9,49.84,not rain,19.5,9.9,1.8,clear-day,Thu,3030
4,7,4.3,76.03,not rain,5.8,9.9,0,clear-day,Sun,1170


In [37]:
citybike_data_2.head()

,hour,temp,dew,humidity,windgust,cloudcover,visibility,solarenergy,uvindex,severerisk,citybike_count
0,6,-1.7,-6.8,67.89,18.3,0,9.9,0,0,10,2153
1,11,10.6,2.7,58.15,18.3,0.4,9.9,1.9,5,10,5300
2,15,6.6,-2.2,53.23,18,99.6,9.9,0.2,1,10,5158
3,7,8.8,6.1,83.17,17.2,100,9.9,0,0,10,1011
4,20,2.2,-9.1,43.05,4.7,28.7,9.9,0,0,10,2217


In [38]:
citybike_data_9.head()

,hour,temp,dew,humidity,windgust,cloudcover,visibility,solarenergy,uvindex,severerisk,citybike_count
0,7,9.9,4.9,71.1,15,99.6,9.9,0,0,10,6336
1,10,10.5,2.6,58.09,18.3,27.5,9.9,1.6,4,10,4990
2,5,9.4,3.3,66,16.1,89.5,9.9,0,0,10,1023
3,16,11.6,0,44.76,8.1,99.6,9.9,0.2,1,10,9250
4,15,12.1,2,50.1,8.1,99.6,9.9,0.6,2,10,7982


## One hot encoding

In [39]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

def one_hot_encode_with_reference(train_df: pd.DataFrame, test_df: pd.DataFrame, columns: list):
    # 初始化编码器
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    
    # 对训练集进行编码
    encoded_train_data = encoder.fit_transform(train_df[columns])
    encoded_train_df = pd.DataFrame(encoded_train_data, columns=encoder.get_feature_names_out(columns))
    train_df_encoded = pd.concat([train_df.drop(columns=columns), encoded_train_df], axis=1)

    # 对测试集进行编码
    encoded_test_data = encoder.transform(test_df[columns])
    encoded_test_df = pd.DataFrame(encoded_test_data, columns=encoder.get_feature_names_out(columns))
    test_df_encoded = pd.concat([test_df.drop(columns=columns), encoded_test_df], axis=1)

    return train_df_encoded, test_df_encoded


In [41]:
taxi_data_9, taxi_data_2 = one_hot_encode_with_reference(taxi_data_9,taxi_data_2, ['hour', 'icon', 'preciptype', 'day_of_week'])

In [44]:
citybike_data_9, citybike_data_2 = one_hot_encode_with_reference(citybike_data_9,citybike_data_2, ['hour'])

## Linear regression model

## Random forest model